In [ ]:
import pandas as pd
import numpy as np

In [ ]:
pip install dask[complete]

In [ ]:
pip install requests

In [ ]:
pip install aiohttp

     |████████████████████████████████| 1.1 MB 5.2 MB/s 
     |████████████████████████████████| 271 kB 45.2 MB/s 
     |████████████████████████████████| 94 kB 2.9 MB/s 
     |████████████████████████████████| 144 kB 47.5 MB/s 


In [ ]:
data_set = []
def load_all_data_from_s3():
  for i in range(1, 13):
    url = f"https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2019-{i}.csv"
    print("reading", i)
    data = pd.read_csv(url)
    data.to_csv('data/yellow_tripdata_2019-{i}.csv')
    print("data set generated for month", i)
load_all_data_from_s3()

In [ ]:
import requests
from tqdm import tqdm
import concurrent.futures

CHUNK_SIZE = 1024 # KB


def get_url(month):
  count = f"0{month}" if month < 10 else f"{month}"
  return f"https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2019-{count}.csv"




def download_file_for_month(month):
  url = get_url(month)
  r = requests.get(url, stream=True)
  file_size = int(r.headers["content-length"])
  with open(f"data/yellow_tripdata_2019-{month}.csv", "wb") as f:
    for data in tqdm(iterable=r.iter_content(chunk_size=CHUNK_SIZE), total=file_size/CHUNK_SIZE, unit="KB"):
      f.write(data)
  print(f"downloaded data set for month {month}")



with concurrent.futures.ThreadPoolExecutor(max_workers=12) as executor:
    executor.map(download_file_for_month, range(1, 13))

In [ ]:
data = pd.read_csv('./data/yellow_tripdata_2019-1.csv')

In [ ]:
data.tip_amount.sum()

In [ ]:
%%time
from re import S
SEQ_SUM = []
SEQ_COUNT = []
def calc_mean(count):
  global SEQ_COUNT
  global SEQ_SUM
  path = f"https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2019-{count if count > 9 else f'0{count}'}.csv"
  print('loading file', count)
  data = pd.read_csv(path)
  data = data[~data.passenger_count.isnull()]
  SEQ_COUNT.append(data.groupby(['passenger_count']).passenger_count.count())
  SEQ_SUM.append(data.groupby(['passenger_count']).tip_amount.sum())


print("calculating mean sequencially")
for i in range(1,13):
  calc_mean(i)
  

MONTHS = ['jan', 'feb', 'mar', 'apr', 'may', 'june', 'july', 'aug', 'sept', 'oct', 'nov', 'dec']

sum_dict = {}
count_dict = {}

for i in range(12):
  sum_dict[MONTHS[i]] = list(SEQ_SUM[i])
  count_dict[MONTHS[i]] = SEQ_COUNT[i]



SEQ_SUM = pd.DataFrame(sum_dict)
SEQ_COUNT = pd.DataFrame(count_dict)

SEQ_SUM['total'] = SEQ_SUM.sum(axis='columns')
SEQ_COUNT['total'] = SEQ_COUNT.sum(axis='columns')

SEQ_MEAN = SEQ_SUM/SEQ_COUNT
SEQ_MEAN.head(10)

In [ ]:
SEQ_MEAN.head(10)

In [ ]:
from dask.distributed import Client

client = Client(n_workers = 4, )

In [ ]:
# %%time
from dask import delayed, compute
import dask.array as da
PAR_SUM = []
PAR_COUNT = []

def calc_mean_parallel(count):
  global PAR_COUNT
  global PAR_SUM
  path = f"https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2019-{count}.csv"
  print('loading file', count)
  data = delayed(pd.read_csv)(path)
  PAR_COUNT.append(delayed(len)(data.index))
  PAR_SUM.append(delayed(data.tip_amount.sum)())

print("calculating mean parallelly")
for i in range(1, 2):
  calc_mean_parallel(i)

PAR_SUM, PAR_COUNT = compute(PAR_SUM, PAR_COUNT)
total_sum = sum(PAR_SUM)
total_count = sum(PAR_COUNT)

def get_mean(sum, count):
  return sum/count

mean = get_mean(total_sum, total_count)
print("calculated mean = ", mean)
client.close()

In [ ]:
import dask.dataframe as dd
import pandas as pd

In [ ]:
%%time
f1 = dd.read_csv([f"https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2019-{count if count > 9 else f'0{count}'}.csv" for count in range(1, 13)], assume_missing=True)

f1['month'] = f1["tpep_pickup_datetime"].map(lambda x: x[5:7])
df2 = f1.groupby(['month', 'passenger_count']).tip_amount.mean().compute().copy()


/usr/local/lib/python3.7/dist-packages/dask/utils.py:30: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  return func(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/dask/utils.py:30: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  return func(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/dask/utils.py:30: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  return func(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/dask/utils.py:30: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  return func(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/dask/utils.py:30: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  return func(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/dask/utils.py:30: DtypeWar

CPU times: user 5min 33s, sys: 23.8 s, total: 5min 57s
Wall time: 4min 15s


In [ ]:

MONTHS = ['jan', 'feb', 'mar', 'apr', 'may', 'june', 'july', 'aug', 'sept', 'oct', 'nov', 'dec']
df2 = df2.unstack().T
df2 = df2.rename(columns = dict(list(zip(f1.columns, MONTHS))))
print(df2)

month                  01        02        03        04        05         06  \
passenger_count                                                                
0.0              1.786901  2.118501  2.164586  2.137596  2.159605   2.173099   
1.0              1.828296  2.166410  2.243850  2.221788  2.269483   2.274122   
2.0              1.833885  2.162850  2.211074  2.207993  2.256493   2.269000   
3.0              1.795886  2.122192  2.167051  2.145546  2.181260   2.200480   
4.0              1.702349  2.030184  2.081816  2.019339  2.055789   2.086528   
5.0              1.869870  2.213183  2.263571  2.252356  2.292741   2.300534   
6.0              1.856687  2.182663  2.231159  2.247027  2.279954   2.289813   
7.0              6.542632  7.917333  5.888750  7.404828  7.079667   5.295745   
8.0              6.480690  5.617500  7.149444  6.810417  6.121290   5.828824   
9.0              3.116667  7.980714  6.064583  5.872632  4.173889  10.318889   

month                   07        08   

In [ ]:
sum_df = f1.groupby(['month', 'passenger_count']).tip_amount.sum().compute().copy()
sum_df = sum_df.unstack().T
sum_df = sum_df.rename(columns = dict(list(zip(f1.columns, MONTHS))))

/usr/local/lib/python3.7/dist-packages/dask/utils.py:30: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  return func(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/dask/utils.py:30: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  return func(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/dask/utils.py:30: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  return func(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/dask/utils.py:30: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  return func(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/dask/utils.py:30: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  return func(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/dask/utils.py:30: DtypeWar

In [ ]:
count_df = f1.groupby(['month', 'passenger_count']).passenger_count.count().compute().copy()
count_df = count_df.unstack().T
count_df = count_df.rename(columns = dict(list(zip(f1.columns, MONTHS))))

/usr/local/lib/python3.7/dist-packages/dask/utils.py:30: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  return func(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/dask/utils.py:30: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  return func(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/dask/utils.py:30: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  return func(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/dask/utils.py:30: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  return func(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/dask/utils.py:30: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  return func(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/dask/utils.py:30: DtypeWar

In [ ]:
sum_df['total'] = sum_df.sum(axis='columns')
count_df['total'] = count_df.sum(axis='columns')
print(sum_df/count_df)

month                  01        02        03        04        05         06  \
passenger_count                                                                
0.0              1.786901  2.118501  2.164586  2.137596  2.159605   2.173099   
1.0              1.828296  2.166410  2.243850  2.221788  2.269483   2.274122   
2.0              1.833885  2.162850  2.211074  2.207993  2.256493   2.269000   
3.0              1.795886  2.122192  2.167051  2.145546  2.181260   2.200480   
4.0              1.702349  2.030184  2.081816  2.019339  2.055789   2.086528   
5.0              1.869870  2.213183  2.263571  2.252356  2.292741   2.300534   
6.0              1.856687  2.182663  2.231159  2.247027  2.279954   2.289813   
7.0              6.542632  7.917333  5.888750  7.404828  7.079667   5.295745   
8.0              6.480690  5.617500  7.149444  6.810417  6.121290   5.828824   
9.0              3.116667  7.980714  6.064583  5.872632  4.173889  10.318889   

month                   07        08   